In [1]:
#from EEGP import EEGPreprocessor
from EGGP_prod import EEGP_prod_processor
import pandas as pd
import numpy as np
from sklearn.utils import resample
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pywt

2023-02-17 17:18:55.104651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("/Users/charleshajjar/Documents/DataAnalyse/data/CaptureSamples/charles_10m-3s_murmure.csv")
df = df.iloc[:300]

In [3]:
df = df[["EEG 1", "EEG 2", "EEG 3", "EEG 4", "EEG 5", "EEG 6", "EEG 7", "EEG 8",
                 "Accelerometer X","Accelerometer Y", "Accelerometer Z", "Gyroscope X",
                 "Gyroscope Y", "Gyroscope Z", "Timestamp"]]

In [4]:
df

,EEG 1,EEG 2,EEG 3,EEG 4,EEG 5,EEG 6,EEG 7,EEG 8,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Timestamp
0,193430.00,31662.80,3786.300,138750.000,-155445.00,112426.000,329340.00,-395224.00,-0.001953,0.925049,-0.260254,0.000000,0.000000,0.000000,24852
1,-11191.30,60044.70,110.507,887.007,47432.60,-6506.950,135470.00,-543017.00,-0.008545,0.941406,-0.272461,0.000000,0.000000,0.000000,91211
2,-51490.90,48005.80,-639.349,164.598,41783.50,-11807.600,89455.50,-205583.00,-0.010742,0.947021,-0.276855,0.000000,0.000000,0.000000,93619
3,-19922.40,38482.80,-4523.010,296.116,34216.70,-11381.300,64256.60,-23497.40,-0.014893,0.952393,-0.278564,0.000000,0.000000,0.000000,95991
4,-5287.71,26011.30,-4219.120,2119.390,21167.10,187.486,45620.30,2509.48,-0.015869,0.956543,-0.281006,-0.183105,-0.152588,-0.244141,148813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3660.14,3636.27,3634.220,3636.540,3648.43,3673.020,3637.25,3656.92,0.014160,0.956299,-0.233154,-2.807620,3.448490,0.122070,1783128
296,3660.14,3636.27,3634.220,3636.540,3648.43,3673.020,3637.25,3656.92,0.014160,0.956299,-0.233154,-2.807620,3.448490,0.122070,1783208
297,3660.14,3636.27,3634.220,3636.540,3648.43,3673.020,3637.25,3656.92,0.014160,0.956299,-0.233154,-2.807620,3.448490,0.122070,1783294
298,3660.14,3636.27,3634.220,3636.540,3648.43,3673.020,3637.25,3656.92,0.014160,0.956299,-0.233154,-2.807620,3.448490,0.122070,1783374


In [5]:
preprocessor = EEGP_prod_processor(df)
filtered_df = preprocessor.apply_butter_filter()
results_df = preprocessor.detect_threshold_changes()
filtered_df = preprocessor.convert_timestamp()
filtered_df = preprocessor.label_thinking_periods(results_df)
transformed_data_df = preprocessor.transform_data(filtered_df)
calculated_p_value_df = preprocessor.calculate_p_value(filtered_df)
p_value_df = filtered_df.assign(p_value=calculated_p_value_df['p_value'])
fdr_corrected_df = preprocessor.fdr_correction(p_value_df)
dwt_grouped_sequences_df = preprocessor.dwt_grouped_sequences_prod(fdr_corrected_df, ["Accelerometer X", "Accelerometer Y",
                      "Accelerometer Z", "Gyroscope X", "Gyroscope Y", "Gyroscope Z"])
df_inter = dwt_grouped_sequences_df
df_final = preprocessor.label_encode_target(df_inter)

/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"EEG Column": column, "Start Time": start_time, "End Time": end_time},
/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"EEG Column": column, "Start Time": start_time, "End Time": end_time},
/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"EEG Column": column, "Start Time": start_time, "End Time": end_time},
/Users/charleshajjar/Documents/DataAnalyse/EEGP.py:92: FutureWarning: The frame.append method is deprecated and will be r

In [6]:
dwt_grouped_sequences_df

,Relative Energy,Target,p_value_fdr,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Mean Spectral Accel X,Mean Spectral Accel Y,Mean Spectral Accel Z,Mean Spectral Gyro X,Mean Spectral Gyro Y
0,189.608123,0,0.682617,0.025970,0.944772,-0.285669,-4.781686,1.641965,5.335191,1.590375,3.421854,11.187439,3.358450,12.183842
1,248.378816,1,0.638330,-0.012394,0.956744,-0.283419,-3.773412,-2.498849,-4.394536,0.689921,2.096957,6.275026,1.644963,6.334659
2,31.967949,0,0.613161,0.038574,0.959229,-0.280273,-4.852290,1.922610,1.434330,0.525083,0.408706,4.282158,1.324436,4.167056
3,39.414372,1,0.613161,-0.006836,0.949870,-0.283366,-3.835043,-1.912433,-0.264485,0.533116,0.267997,1.972918,0.437749,2.220173
4,40.071486,0,0.536450,0.039811,0.957178,-0.278027,-5.389405,2.126058,1.599122,1.073788,1.256595,9.628869,2.979719,9.488232
5,1.025459,1,0.536450,-0.015869,0.956543,-0.281006,-0.183105,-0.152588,-0.244141,0.299067,0.724000,0.239351,0.055276,0.004428
6,149.128970,0,0.505458,0.020392,0.942421,-0.282445,-3.135280,0.795064,5.772641,1.206048,1.487048,6.746547,1.883548,7.625255
7,42.974010,1,0.505458,-0.008301,0.944824,-0.283203,-4.211423,-2.003987,0.569662,0.555558,0.402165,1.913191,0.393086,2.294821
8,81.720832,0,0.464243,0.014357,0.938812,-0.277012,-2.945629,-0.097019,3.665755,1.224478,2.521233,11.983932,3.822892,11.600300
9,0.884869,1,0.464243,-0.001953,0.925049,-0.260254,0.000000,0.000000,0.000000,0.287838,0.732860,0.170500,0.038424,0.062715


In [11]:
df_final

,Relative Energy,Target,p_value_fdr,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Mean Spectral Accel X,Mean Spectral Accel Y,Mean Spectral Accel Z,Mean Spectral Gyro X,Mean Spectral Gyro Y
0,189.608123,0,0.682617,0.025970,0.944772,-0.285669,-4.781686,1.641965,5.335191,1.590375,3.421854,11.187439,3.358450,12.183842
1,248.378816,1,0.638330,-0.012394,0.956744,-0.283419,-3.773412,-2.498849,-4.394536,0.689921,2.096957,6.275026,1.644963,6.334659
2,31.967949,0,0.613161,0.038574,0.959229,-0.280273,-4.852290,1.922610,1.434330,0.525083,0.408706,4.282158,1.324436,4.167056
3,39.414372,1,0.613161,-0.006836,0.949870,-0.283366,-3.835043,-1.912433,-0.264485,0.533116,0.267997,1.972918,0.437749,2.220173
4,40.071486,0,0.536450,0.039811,0.957178,-0.278027,-5.389405,2.126058,1.599122,1.073788,1.256595,9.628869,2.979719,9.488232
5,1.025459,1,0.536450,-0.015869,0.956543,-0.281006,-0.183105,-0.152588,-0.244141,0.299067,0.724000,0.239351,0.055276,0.004428
6,149.128970,0,0.505458,0.020392,0.942421,-0.282445,-3.135280,0.795064,5.772641,1.206048,1.487048,6.746547,1.883548,7.625255
7,42.974010,1,0.505458,-0.008301,0.944824,-0.283203,-4.211423,-2.003987,0.569662,0.555558,0.402165,1.913191,0.393086,2.294821
8,81.720832,0,0.464243,0.014357,0.938812,-0.277012,-2.945629,-0.097019,3.665755,1.224478,2.521233,11.983932,3.822892,11.600300
9,0.884869,1,0.464243,-0.001953,0.925049,-0.260254,0.000000,0.000000,0.000000,0.287838,0.732860,0.170500,0.038424,0.062715


In [7]:
#df_final = preprocessor.label_encode_target(dwt_grouped_sequences_df)

In [8]:
df_final.Target.value_counts()

0    7
1    7
Name: Target, dtype: int64

In [12]:
# Triez les données selon le critère "Target" et "Relative Energy"
df_final = df_final.loc[df_final["Relative Energy"] >= 18]
df_final = df_final.sort_values(by=["Target", "Relative Energy"], ascending=[True, False])
df_final = df_final.reset_index(drop=True)

In [13]:
df_final

,Relative Energy,Target,p_value_fdr,Accelerometer X,Accelerometer Y,Accelerometer Z,Gyroscope X,Gyroscope Y,Gyroscope Z,Mean Spectral Accel X,Mean Spectral Accel Y,Mean Spectral Accel Z,Mean Spectral Gyro X,Mean Spectral Gyro Y
0,189.608123,0,0.682617,0.025970,0.944772,-0.285669,-4.781686,1.641965,5.335191,1.590375,3.421854,11.187439,3.358450,12.183842
1,149.128970,0,0.505458,0.020392,0.942421,-0.282445,-3.135280,0.795064,5.772641,1.206048,1.487048,6.746547,1.883548,7.625255
2,134.022092,0,0.464243,0.015550,0.947322,-0.277156,-4.730226,0.133809,4.758395,1.084299,1.578420,7.154655,2.212042,7.186261
3,81.720832,0,0.464243,0.014357,0.938812,-0.277012,-2.945629,-0.097019,3.665755,1.224478,2.521233,11.983932,3.822892,11.600300
4,40.071486,0,0.536450,0.039811,0.957178,-0.278027,-5.389405,2.126058,1.599122,1.073788,1.256595,9.628869,2.979719,9.488232
5,31.967949,0,0.613161,0.038574,0.959229,-0.280273,-4.852290,1.922610,1.434330,0.525083,0.408706,4.282158,1.324436,4.167056
6,27.279803,0,0.464243,0.018685,0.952035,-0.247852,-3.381349,2.301027,1.045736,1.000537,0.299796,11.473074,3.836095,10.138521
7,248.378816,1,0.638330,-0.012394,0.956744,-0.283419,-3.773412,-2.498849,-4.394536,0.689921,2.096957,6.275026,1.644963,6.334659
8,42.974010,1,0.505458,-0.008301,0.944824,-0.283203,-4.211423,-2.003987,0.569662,0.555558,0.402165,1.913191,0.393086,2.294821
9,39.414372,1,0.613161,-0.006836,0.949870,-0.283366,-3.835043,-1.912433,-0.264485,0.533116,0.267997,1.972918,0.437749,2.220173
